In [4]:
# 1) load packages env py_dl
import os
# os.environ['OMP_NUM_THREADS'] = '50'
import numpy as np
import pandas as pd 
import random
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv1D
from keras import initializers


In [5]:
# 2) constants and hyperparameters
L_RBNS = 20 # length of each sequence in RBNS data
O = int(1e7) # for initializing big arrays, helps reduce runtime
LIMIT_FILE_N_SEQ_READ = int(1e6) # limit the amount of seq we read from file, helps reduce runtime
ONE_HOT_DICT = {b'A': np.array([0,0,0,1], dtype=np.float16),
                b'C': np.array([0.,0.,1,0], dtype=np.float16),
                b'G': np.array([0,1,0,0], dtype=np.float16),
                b'T': np.array([1,0,0,0], dtype=np.float16),
                b'U': np.array([1,0,0,0], dtype=np.float16),
                b'N': np.array([0.25,0.25,0.25,0.25])
                }

ONE_HOT_DICT2 = {'A':[0,0,0,1],
                'C': [0,0,1,0],
                'G':[0,1,0,0],
                'T': [1,0,0,0],
                'U': [1,0,0,0],
                'N': [0.25,0.25,0.25,0.25]
                }

L_RBNS = 20 # length of each sequence in RBNS data
FILES_40 = ["RBP9" , "RBP11", "RBP15", "RBP31"] # files  with sequences len 40 and not 20

model_param_dict = {"kernel_size":3, "pool_size":2, "layers": [128, 128], "final_activation_function":"sigmoid"}

In [9]:
# 3) read files 
# list files in RBNS_training
directory = '/data01/private/resources/RACHELI_EDEN_SHARED/DL_PROJ/RBNS_training'
RBNS_training_files = []
for filename in os.listdir(directory): # Iterate over all files in the directory
    if os.path.isfile(os.path.join(directory, filename)):
        RBNS_training_files.append(filename)
# print(RBNS_training_files)

# list files in single protein - filter out protein 1 files only
PROTEIN = "RBP11" ### TODO change 
filtered_list = [value for value in RBNS_training_files if value.startswith(str(PROTEIN)+'_')]
print(filtered_list)

# order file names according to concentration
modified_list = [value.replace(f'{PROTEIN}_', '').replace('nM.seq', '') for value in filtered_list]
if f'{PROTEIN}_input.seq' in filtered_list: # get rid of input so only ints are left
    modified_list.remove('input.seq')
modified_list = sorted(modified_list, key=int)
modified_list = modified_list[:3]+modified_list[-3:]
modified_list = [PROTEIN+"_"+str(modified_list[i])+"nM.seq" for i in range(len(modified_list))]
modified_list.insert(0,str(PROTEIN)+"_input.seq")
print(modified_list)


['RBP11_1090nM.seq', 'RBP11_121nM.seq', 'RBP11_13nM.seq', 'RBP11_1nM.seq', 'RBP11_3280nM.seq', 'RBP11_365nM.seq', 'RBP11_40nM.seq', 'RBP11_4nM.seq', 'RBP11_9800nM.seq', 'RBP11_input.seq']
['RBP11_input.seq', 'RBP11_1nM.seq', 'RBP11_4nM.seq', 'RBP11_13nM.seq', 'RBP11_1090nM.seq', 'RBP11_3280nM.seq', 'RBP11_9800nM.seq']


In [4]:
# 4) MULTICALSS classification - 
# initalize np.arrays
N = LIMIT_FILE_N_SEQ_READ * len(modified_list) # number of seqs
master_list = np.empty((N), dtype=f'|S{L_RBNS}') # sequences
class_lables = np.zeros((N, len(modified_list)), dtype=np.int8) # array of probolities per class
class_lables[:, 0] = 1 # set all to 1 for the first class (input.seq)
for i, file in enumerate(modified_list): # set the lables for the rest of the classes
    if file != str(PROTEIN + '_input.seq'):
        class_lables[LIMIT_FILE_N_SEQ_READ * i: LIMIT_FILE_N_SEQ_READ * (i + 1), i:] = 1
n = 0 # index for master_list
## running on protein 1 files only ############################################################
for file_index, file in enumerate(modified_list):
    #print(file_index)
    file_path = '/data01/private/resources/RACHELI_EDEN_SHARED/DL_PROJ/RBNS_training/' + file
    with open(file_path, 'r') as f:
        lines = [f.readline() for _ in range(LIMIT_FILE_N_SEQ_READ)] # read LIMIT_FILE_N_SEQ_READ lines
    # cut seq to 20 length randomly if len is longer find a random starting index and take 20 from there
    if file in FILES_40: # file len 40
        # choose random index to start from and select 20 chars - save shortend seq into master_list
        start_index = random.randint(0, 20)
        for line in lines:
            seq = line.split('\t')[0] # take only the sequence
            master_list[n] = seq[start_index : start_index + 20] # shortened seq
            n += 1
    else: # file len 20 + isnt input.seq
        for line in lines:
            seq = line.split('\t')[0] # take only the sequence
            master_list[n] = seq # seq
            n += 1
del lines

In [5]:
# 
random.seed(123)
rand_master_list = np.empty((LIMIT_FILE_N_SEQ_READ * 2), dtype=f'|S{L_RBNS}') # sequences
rand_class_labels = np.zeros((LIMIT_FILE_N_SEQ_READ * 2, len(modified_list)), dtype=np.int8)
ABC = [b'A', b'C', b'G', b'T']
rand_master_list = np.array([b''.join([ABC[random.randint(0,3)] for _ in range(L_RBNS)]) for _ in range(LIMIT_FILE_N_SEQ_READ * 2)], dtype=f'|S{L_RBNS}')

In [6]:
class_lables = np.concatenate((class_lables, rand_class_labels))
master_list = np.concatenate((master_list, rand_master_list))

In [7]:
PAD_SIZE = model_param_dict["kernel_size"] - 1
SEQ_PADDED_LEN = 20 + 2 * PAD_SIZE # TODO: change 20 to 20 or 40
one_hot = np.array([[ONE_HOT_DICT[bytes([nuc])] for nuc in (b"N" * PAD_SIZE + seq + b"N" * PAD_SIZE)] for seq in master_list])

In [8]:
# 5) NN per protien - input each seq (len 20) in master_list into NN 
    # output = vec of probabilities (one per each concentration)
    # compare output to bool numpy array (true lable)
    # backpropogation to model to minimize loss?
import tensorflow as tf

# Define the metric for model
def accuracy_th(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)  # Cast labels to float32
    threshold = 0.5
    y_pred_thresholded = tf.where(y_pred >= threshold, 1., 0.)  # Apply the threshold to the predicted probabilities
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_true, y_pred_thresholded), tf.float32))  # Calculate the accuracy
    return accuracy


In [ ]:

output_size = 6 # number of classes
X = one_hot
Y = class_lables

#  CNN
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=model_param_dict["kernel_size"], strides=1,
                 kernel_initializer=initializers.RandomNormal(stddev=0.01), activation='relu',
                 input_shape=X.shape[1:], use_bias=True, bias_initializer='RandomNormal'))
model.add(Conv1D(filters=128, kernel_size=model_param_dict["kernel_size"], strides=3))
# model.add(MaxPooling1D(pool_size=model_param_dict["pool_size"], strides=None, padding='valid', data_format='channels_last')) # MAYBE USE CONV KERNEL HERE INSTEAD 
model.add(Flatten())
model.add(Dropout(0.1))
# per layer
for layer_size in model_param_dict['layers']:
            print(f'the layer size is: {layer_size}')
            model.add(Dense(layer_size, activation='relu'))
            model.add(Dropout(0.1))
model.add(Dense(output_size, activation=model_param_dict['final_activation_function'])) 

from keras.optimizers import Adam, SGD, RMSprop
# Hyperparameters
EPHOCHS = 3
model.compile(optimizer=Adam(learning_rate = .003), loss="binary_crossentropy", metrics=[accuracy_th])
model.summary()

# shuffle data
rng = np.random.default_rng(32)
shuf_inds = rng.permutation(X.shape[0])
X = X[shuf_inds][:int(1e6)]
Y = Y[shuf_inds][:int(1e6)]

# Compile the model with Optimizer, Loss Function and Metrics
run_hist_1 = model.fit(X, Y, batch_size=256, epochs=EPHOCHS, validation_split=0.5, shuffle=True, use_multiprocessing=True, workers=90)
model.save('/data01/private/resources/RACHELI_EDEN_SHARED/DL_PROJ/model_multiclass.h5') # save the model

the layer size is: 128
the layer size is: 128
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 22, 128)           1664      
                                                                 
 conv1d_1 (Conv1D)           (None, 7, 128)            49280     
                                                                 
 flatten (Flatten)           (None, 896)               0         
                                                                 
 dropout (Dropout)           (None, 896)               0         
                                                                 
 dense (Dense)               (None, 128)               114816    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (

2023-07-31 22:53:40.574429: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/3
1954/1954 [==============================] - 234s 119ms/step - loss: 0.5777 - accuracy_th: 0.6970 - val_loss: 0.5752 - val_accuracy_th: 0.6987
Epoch 2/3
1954/1954 [==============================] - 224s 115ms/step - loss: 0.5757 - accuracy_th: 0.6983 - val_loss: 0.5752 - val_accuracy_th: 0.6987
Epoch 3/3
1954/1954 [==============================] - 220s 113ms/step - loss: 0.5755 - accuracy_th: 0.6984 - val_loss: 0.5752 - val_accuracy_th: 0.6989


PermissionError: [Errno 13] Unable to create file (unable to open file: name = '/data01/private/resources/RACHELI_EDEN_SHARED/DL_PROJ/model_multiclass.h5', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 242)

In [9]:
# load the model
model = keras.models.load_model("/data01/private/resources/RACHELI_EDEN_SHARED/DL_PROJ/model_multiclass.h5", custom_objects={"accuracy_th":accuracy_th})

2023-08-01 12:01:56.476668: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-01 12:01:56.498853: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
# 6) # cut seg from RNAcomplete file into all shifts of len 24
# read txt file
file_path = '/data01/private/resources/RACHELI_EDEN_SHARED/DL_PROJ/RNAcompete_sequences.txt'
with open(file_path, 'r') as file:
    # convert seqences to one hot encoding + pad seq 
    RNAcompete_master_list_one_hot = [[ONE_HOT_DICT2[nuc] for nuc in ((41 - len(seq.rstrip('\n'))) * 'N' + seq.rstrip('\n'))] for seq in file if len(seq) > 5]
num_seqs = len(RNAcompete_master_list_one_hot)

# SHIFTS - RNAcomplete file into all shifts of len 24 -- create k-mers of all possible shifts 
shifts_RNAcompete_master_list = [[seq[i:i+SEQ_PADDED_LEN] for i in range(0, len(seq) - SEQ_PADDED_LEN + 1, 5)] for seq in RNAcompete_master_list_one_hot] # step +5 
shifts_RNAcompete_master_list = np.reshape(shifts_RNAcompete_master_list, (-1, SEQ_PADDED_LEN, 4))

In [11]:
# create features: MULTICLASS parallelize the prediction + output -  suppose predict gives N dim vector:
N = 6 # number of classes
prediction = model.predict(shifts_RNAcompete_master_list, use_multiprocessing=True, workers=90)
prediction = np.reshape(prediction, (num_seqs, -1, N))
# fetures - 6 max + 6 min per seq with best shift
features = np.empty((num_seqs, 2 * N))
features[:, :N] = np.max(prediction, axis=1) # shape is (num_seqs, N)
features[:, N:] = np.min(prediction, axis=1) # shape is (num_seqs, N)

# # write pred into file
# protein="RBP1"
# with open(f'/data01/private/resources/RACHELI_EDEN_SHARED/DL_PROJ/max_predicitions_{protein}.txt', "w") as file:
#     for pred in max_prediction:
#         file.write(f'{pred}\n')

30170/30170 [==============================] - 216s 7ms/step


Mix proteins

In [45]:
# # LinearRegression - between y_hat and y_target
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
# X = np.array(features)  # Independent variable (features)

# with open('/data01/private/resources/RACHELI_EDEN_SHARED/DL_PROJ/RNCMPT_training/RBP1.txt','r') as file:
#     y_target=[float(line.strip()) for line in file if line != ""] ### y

# y = np.array(y_target)  # Dependent variable (target)

# model = LinearRegression()
# model.fit(X, y)
# y_pred = model.predict(X)

# from scipy.stats import pearsonr
# correlation, _ = pearsonr(y, y_pred)
# print(correlation)

# correlation_coefficient, p_value = stats.pearsonr(y_target, features_max)
# print("Pearson correlation coefficient:", correlation_coefficient, "p-value:", p_value)

# # Print the coefficients (slope and intercept)
# print("Coefficients:", model.coef_)     # Slope
# print("Intercept:", model.intercept_)   # Intercept
# # print("Maximum:", max(y_pred))
# # print("Minimum:", min(y_pred))

# # Visualize the linear regression line
# plt.scatter(X, y) 
# plt.plot(X, y_pred, color='red', label='Linear Regression') ## model
# plt.show()

0.27241164380909


In [47]:
# # LinearRegression - Get the coefficients and the intercept
# coefficients = model.coef_
# intercept = model.intercept_

# # Create the formula for the Linear Regression model
# formula_parts = [f"{coeff:.4f}" for coeff in coefficients]
# formula = f"y = {intercept:.4f} + " + " + ".join(formula_parts)

# print("Linear Regression Formula:")
# print(formula)

Linear Regression Formula:
y = 1.1906 + -15.2344 + 20.3540 + -9.8116 + -9.1625 + -1.7815 + 24.2165 + -4.4641 + -15.8454 + 11.6378 + -11.0284 + 2.0439 + 8.7680


In [12]:
with open('/data01/private/resources/RACHELI_EDEN_SHARED/DL_PROJ/RNCMPT_training/RBP1.txt','r') as file:
    y_target=np.array([float(line.strip()) for line in file if line != ""])

# # Create a Sequential model with 1 output neurons for Pearson correlation
output_neurons = 1
model = Sequential()
model.add(Dense(1, activation='linear', input_shape=(features.shape[1],))) # single neuron layer with 'linear' activation (no activation)

# Define the custom Pearson correlation coefficient loss function
def pearson_correlation_loss(y_true, y_pred):
    return 1 - pearson_correlation(y_true, y_pred)  # Return 1 - correlation to minimize the negative correlation

def pearson_correlation(y_true, y_pred):
    y_true_mean = tf.reduce_mean(y_true)
    y_pred_mean = tf.reduce_mean(y_pred)
    covariance = tf.reduce_mean((y_true - y_true_mean) * (y_pred - y_pred_mean))
    y_true_std = tf.math.reduce_std(y_true)
    y_pred_std = tf.math.reduce_std(y_pred)
    correlation = covariance / (y_true_std * y_pred_std)
    return correlation  # Return 1 - correlation to minimize the negative correlation

# Compile model with the custom loss function
model.compile(optimizer='adam', loss=pearson_correlation_loss, metrics=[pearson_correlation])
# Fit the model to the data
model.fit(features, y_target, batch_size=128, epochs=5, validation_split=0.5)

Epoch 1/5
943/943 [==============================] - 2s 2ms/step - loss: 0.6517 - pearson_correlation: 0.3483 - val_loss: 0.6263 - val_pearson_correlation: 0.3737
Epoch 2/5
943/943 [==============================] - 1s 1ms/step - loss: 0.6254 - pearson_correlation: 0.3746 - val_loss: 0.6210 - val_pearson_correlation: 0.3790
Epoch 3/5
943/943 [==============================] - 4s 4ms/step - loss: 0.6231 - pearson_correlation: 0.3769 - val_loss: 0.6189 - val_pearson_correlation: 0.3811
Epoch 4/5
943/943 [==============================] - 5s 5ms/step - loss: 0.6219 - pearson_correlation: 0.3781 - val_loss: 0.6155 - val_pearson_correlation: 0.3845
Epoch 5/5
943/943 [==============================] - 5s 5ms/step - loss: 0.6183 - pearson_correlation: 0.3817 - val_loss: 0.6171 - val_pearson_correlation: 0.3829


In [13]:
weights, biases = model.layers[0].get_weights()
# Create the formula for the Linear Regression model
formula_parts = [f"{coeff}" for coeff in weights]
formula = f"y = {biases[0]} (b) + " + " + ".join(formula_parts)

print("NN Formula:")
print(formula)

NN Formula:
y = -0.011234917677938938 (b) + [0.10428984] + [0.501944] + [-0.3563016] + [-0.52175224] + [-0.59814584] + [0.7204223] + [0.18594183] + [0.04441676] + [-0.46729964] + [-0.7657383] + [0.30190748] + [0.11686829]


In [ ]:
# 4) model feature to find RNCMPT_train on all proteins